# Data Ingestion

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [2]:
llm = ChatOpenAI(
    model = "arcee-ai/trinity-large-preview:free",
    temperature = 0.9,
    base_url = "https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "http://localhost:8000",
        "X-Title": "RAG with Langchain"
    }
)

In [6]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun 

In [7]:
arxiv_api_wrapper = ArxivAPIWrapper(top_k=2, doc_content_chars_max=300)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

In [8]:
from langchain_community.document_loaders import WikipediaLoader

In [9]:
def load_wikipedia_data(query_topic, max_docs=2):
    loader = WikipediaLoader(query=query_topic, load_max_docs=max_docs)
    docs = loader.load()
    return docs

# Embeddings and Vector Store

In [ ]:
from langchain_cohere import CohereEmbeddings
from langchain_astradb import AstraDBVectorStore
import os

In [ ]:
embeddings = CohereEmbeddings(
		model="embed-english-light-v3.0",
		cohere_api_key=os.getenv("COHERE_API_KEY"),
	)
Vectorstore = AstraDBVectorStore(
    collection_name = "langchain_tutorial",
    embedding= embeddings,
    token= os.getenv("ASTRA_DB_APLICATION_TOKEN")
)

In [ ]:
print("AstraDB Vector Store initialized successfully")

In [ ]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun 

In [ ]:
query = "What is Retrieval Augmented Generation"

In [ ]:
result = vectorstore.similarity_search(query)
result

In [ ]:
result[0].page_content

In [ ]:
result[1].page_content

In [ ]:
result[2].page_content

In [ ]:
result[3].page_content

# Retriever and Chain In Langchain

In [ ]:
model = ChatOpenAI(
    model = "arcee-ai/trinity-large-preview:free",
    temperature = 0.9,
    base_url = "https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "http://localhost:8000",
        "X-Title": "RAG with Langchain"
    }
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt1 = ChatPromptTemplate.from_template("""
    Answer the Following Question based on the Provided Context only.
    Make sure the answer is satisfactiable.
    <context>
    {context}
    </context>
    Question: {input}                        
    """)

## Chain

In [ ]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain


In [ ]:
chain_document = create_stuff_documents_chain(model, prompt1)

## Retriver

In [ ]:
retriever = vectorstore.as_retriever()
retriever

In [ ]:
# Retriver with Chain
from langchain_classic.chains import create_retrieval_chain

In [ ]:
retreival_chain = create_retrieval_chain(retriever, chain_document)

In [ ]:
response = retreival_chain.invoke({"input": "What is Retrieval Augmented Generation"})
response

In [ ]:
response['answer']